#### Welcome to the Parallel C OpenMP hand-on session. 

#### In this Jupyter Notebook, you are provided with four scientific programming tasks that can be accelerated by OpenMP multithreading. Each example highlights a distinct loop-parallelisation theme and showcases different strategies.

In [ ]:

import os
# the jupyter notebook is launched from your $HOME, change the working directory provided a username directory is created under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/OpenMP-C"))



# Exercise 1. (Independent Loop Iterations) Monte-Carlo $\pi$

Consider the quarter of a unit circle inscribed in the unit square $[0,1]^2$. Draw $N$ points uniformly at random in the square, and let $h$ be the number that fall inside the quarter circle $(x^2+y^2\le 1)$. By the area ratio,
\begin{align*}
\hat{\pi} \approx \frac{4h}{N}.
\end{align*}

Increasing $N$ reduces the estimator’s variance (error estimate is $O(N^{-1/2})$). Because each sample is independent, this Monte Carlo approach is *embarrassingly parallel*, making it an ideal example for OpenMP Multithreading paralelisation.



Note that in the python-based Jupyter environment you are working on, we need to prepend lines of code with `!` to indicate we wish to execute a shell command.

### Exercise 1.0
Inspect the serial codebase [monte-carlo-pi-serial.c](./monte-carlo-pi-serial.c). Idenity the for-loop where OpenMP multithreading is suitable.

In [29]:
# Compilee the serial code
!gcc -g -Wall -O3 -o monte-carlo-pi-serial monte-carlo-pi-serial.c -lm

In [ ]:
%%time
#Execute the serial code
!./monte-carlo-pi-serial

### Exercise 1.1: 
Add OpenMP work-sharing loop constructs and appropriate clauses to [monte-carlo-pi-omp.c](./monte-carlo-pi-omp.c) to parallelise the computation.

In [ ]:
# Compile the openmp code
!gcc -fopenmp -g -Wall -O3 -o monte-carlo-pi-omp monte-carlo-pi-omp.c -lm

In [ ]:
%%time

# Execute the openmp code, compare the time consumption with the serial version
!OMP_NUM_THREADS=4 ./monte-carlo-pi-omp

### Takeaway from Exercise 1:
If your workload is embarrassingly parallel (independent iterations, little/no communication), OpenMP is an excellent first choice on a single shared-memory machine.

# Exercise #2 (Dynamic Loop Iterations) Mandelbrot Set
## Mandelbrot Set

The **Mandelbrot set** is the set of complex numbers $c \in \mathbb{C}$ for which the sequence

$$z_{n+1} = z_n^2 + c,\qquad z_0 = 0$$
remains **bounded** (does not diverge to infinity).

We write
$$
M \;=\; \Big\{\, c \in \mathbb{C} \;\big|\; \{z_n\}_{n\ge 0}\ \text{is bounded for } z_{n+1}=z_n^2+c,\ z_0=0 \,\Big\}.
$$

### Convergence (Escape) Test
For visualization, we sample $c$ on a uniform grid over the rectangle
$$
[-2.0,\ 0.47] \times [-1.12,\ 1.12]\,i,
$$
which covers the most visually interesting portion of $M$.

For each grid point $c$, iterate the map up to $N_{\max}=100$ steps:
$$
z_{n+1} = z_n^2 + c,\quad n=0,\dots, N_{\max}-1.
$$
If at any step $|z_n| > 2$ (equivalently $|z_n|^2 > 4$), the orbit is guaranteed to diverge and $c \notin M$. If $|z_n|$ never exceeds the value $2$ within $N_{\max}$ steps, we treat $c$ as (numerically) belonging to $M$.

### Dynamic Workload
Different points require different numbers of iterations before diverging (some escape in a few steps, others never escape within $N_{\max}$). This **variable iteration count** creates load imbalance in parallel implementations. To mitigate this with OpenMP, use **dynamic scheduling** (e.g., `schedule(dynamic, chunk)`) so threads pick up new points as they finish, improving overall utilization.

 

### Exercise 1.0:
Inspect the serial codebase [mandelbrot-serial.c](./mandelbrot-serial.c).
Compile and run the serial version of mandelbrot set generator.

In [ ]:
# Compile the serial code

!gcc -Wall -O3 mandelbrot-serial.c -o mandelbrot-serial

In [ ]:
# Execute the serial program 
!./mandelbrot-serial

In [ ]:
# Plot mandelbrot set to verify outputs
%run  'Mandelbrot-plot.py'

### Exercise 2.1
Add OpenMP work-sharing loop constructs and appropriate clauses to [mandelbrot-omp.c](./mandelbrot-omp.c) to parallelise the computation. Ensure that writes to the output file remain in order (e.g., with #pragma omp ordered).

In [ ]:
# Compile the openmp code
!gcc -fopenmp -g -Wall -O3 -o mandelbrot-omp mandelbrot-omp.c 

In [ ]:
%%time
# Execute the openmp code

!OMP_NUM_THREADS=4  ./mandelbrot-omp

In [ ]:
#  Verify the results by plot
%run  'Mandelbrot-plot.py'

You’ve probably noticed the OpenMP version doesn’t run faster. The culprit is the use of `ordered` around your output.

**What `ordered` does:**  
`ordered` enforces that a specific block inside a `parallel for` executes **in the original loop order**. Only one iteration at a time may enter that block. In other words, it **serializes** the part it wraps.

**What to do instead (best practice)**
- **Keep compute parallel, do I/O serially.**  
  Compute the iteration counts into a preallocated buffer in the `parallel for` and write the data after the parallel region.


### Exercise 2.2 
Modify [mandelbrot-separateIO.c](./mandelbrot-separateIO.c) to separate the I/O from the computation routine. 

In [28]:
# compile the openmp code after IO separation
!gcc -fopenmp -g -Wall -O3 -o mandelbrot-separateIO mandelbrot-separateIO.c 


In [ ]:
%%time
 
!OMP_NUM_THREADS=4  ./mandelbrot-separateIO

In [ ]:
# Verify the output again by plot
%run  'Mandelbrot-plot.py'

### Takeaway from Exercise 2:
When planning acceleration, consider refactoring the code to fit your parallelisation tools. Here, we separated I/O from computation so the compute kernel could be parallelised cleanly with OpenMP (no I/O inside parallel regions).

# Exercise #3 (Loop Dependence). Solve Linear Equation by Conjugate Gradient Method 

The Conjugate Gradient method is a numerical method that is wildely used in solving certain type of matrix problems. It is also the base ingredient of [HPCG Benchmark](https://www.hpcg-benchmark.org/) for ranking HPC systems.


Consider solving a linear equation
\begin{align*}
Ax = b
\end{align*}
where matrix $A \in \mathbf{R}^{n\times n}$ is symmetric positive definite. 

The initial guess $x_0$ can be any approximation, we choose $0$. The baseline algorithm is statedd as following:

Compute $r_0 = b - Ax_0$

For $i= 0, \cdots, n$ Do 

 $\alpha_i := (r_i, r_i)/(Ap_i, p_i)$\;
 
 $x_{i+1}:=x_i+\alpha_i p_i$\;

 $r_{i+1}:=r_i -\alpha_i Ap_i$\;
 
 If $r_{i+1} <\text{tolerance}$ Then Break
 
 $\beta_i:= (r_{i+1}, r_{i+1}) / (r_i, r_i)$\;
 
 $p_{i+1}:= r_{i+1} +\beta_i p_i$

Conjugate Gradient (CG) method is a direct method that produces the exact solution at most $n$ steps, however, in practice a tolerance is usually set to terminate iterations.

CG method guarantees convergence for symmetric positive definite matrices in theory.



### Parallelisation in CG method
Note that the for-loop in CG method possesses the dependence between iterations. With multiple threads, each iteration can only be execute only after the dependence is met leading to a non-parallelizable loop. 

That being said, we can still use OpenMP to parallelise part of the code after analysing the bottleneck of its performance!

### Test Matrices
For numerical experiments, we will test two matrices [Trefethen_20](https://www.cise.ufl.edu/research/sparse/matrices/JGD_Trefethen/Trefethen_20.html) and [Msc04515](https://www.cise.ufl.edu/research/sparse/matrices/Boeing/msc04515.html).

Trefethen_20 is a small-sized problem in which you should see a fast convergence.

Msc04515 is a real-life problem arising from a structural engineering. It is an ill-conditioned matrix, which essentially means hard to solve and requires a lot more iterations for CG method if it converges at all.

### Exercise 3.0
Inspect the serial code base [cg-std.c](./cg-std.c), and map each step of the pseudo-algorithm to its corresponding implementation in the code

In [ ]:
# Compile the serial code

!gcc -g -Wall -O3 -o cg-std cg-std.c -lm

In [ ]:
# Execute the serial program to solve Trefethen_20 problem
!./cg-std 1e-5 < Trefethen_20.dat

### Exercise 3.1
Add the appropriate OpenMP work-sharing constructs and clauses to the loops in [cg-std-omp.c](./cg-std-omp.c) to  to parallelise execution and improve performance.

Once you are one, compile the code with the following command.


In [ ]:
# Compile the openmp code
!gcc -fopenmp -g -Wall -O3 -o cg-std-omp cg-std-omp.c -lm

Test your code with a simple matrix [Trefethen_20.dat](./Trefethen_20.dat)

In [ ]:
%%time
# Verify the numerical result with the serial version
!OMP_NUM_THREADS=4  ./cg-std-omp 1e-5 < Trefethen_20.dat

If the result looks good, test it with a more serious matrix [msc04515.dat](./msc04515.dat)

In [ ]:
%%time
# Excute the parallel code to solve Boeing msc04515 problem. You may also want to attempt this problem with the serial code

!OMP_NUM_THREADS=4 ./cg-std-omp 1e-5 < msc04515.dat

Compare the results with a serial code.

In [ ]:
%%time
# Excute the parallel code to solve Boeing msc04515 problem. You may also want to attempt this problem with the serial code

!OMP_NUM_THREADS=4 ./cg-std 1e-5 < msc04515.dat

### Takeaway from Exercise 3:
You don’t always need to understand the entire codebase (though it often helps). Even in a large project, pinpointing the bottleneck and adding targeted parallelism can deliver significant speedups.

# Exercise #4 (Parallelisaton vs. Convergence Rate): Solve Finite Difference Discretised Poisson Equation by Jacobi and Gauss-Seidel Methods


### Model Problem
Consider a 2D Poisson equation with Dirichlet boundary condition over a unit square domain $\Omega = [0,1] \times [0,1]$

\begin{align*}
-\Delta u &= f \; \text{in} \; \Omega \\
 u &= g \; \text{on} \; \partial \Omega
\end{align*}

Define a uniform partition of the domain $\Omega$ with nodal points at which the solution of the Poisson equation is sampled. Let $h$ be the uniform distance between two nodal points then the nodal points that lie on the mesh are defined by

\begin{align*}
x_i = i h, \; y_j = j h\qquad i,j = 0,\cdots, N
\end{align*}
where $N$ is a given mesh size and $i, j$ are integers along $x, y$-axis telling the location of each nodal point.  


### Discretisation
We use the second-order central finite difference method to discretise the Laplace operator
\begin{align*}
(\Delta u)_{i,j}
  &= \bigl(D_{xx}^2 u\bigr)_{i,j} + \bigl(D_{yy}^2 u\bigr)_{i,j} \\[2ex]
  &\approx \frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{h^2}
          + \frac{u_{i,j+1}-2u_{i,j}+u_{i,j-1}}{h^2}.
\end{align*}
leading to
\begin{align}
    -(\Delta u)_{i,j} = \frac{4u_{i,j}-u_{i+1,j}-u_{i-1,j}-u_{i,j+1}-u_{i,j-1}}{h^2}=f(u_{i,j}).
\end{align}

The above finite-difference formula can further be represented by a five-point stencil matrix built in the mesh
\begin{align*}S = 
    \begin{pmatrix}
    & -1 & \\
    -1 & 4 &-1\\
    & -1 &
    \end{pmatrix}.
\end{align*}

Impose the Dirichlet boundary condition, on the interior nodal points the discretisation can be written as a linear equation

\begin{align*}
A u = f, \qquad A=\frac{1}{h^2}
    \begin{pmatrix}
S & I \\
I & S & I \\
& I & \ddots & \ddots \\
& & \ddots & \ddots & I \\
& & & I & S
\end{pmatrix}
\end{align*}
where $A \in \mathbb{R}^{(N-2)^2 \times (N-2)^2}$, $u \in \mathbb{R}^{(N-2)^2}$ and $f \in \mathbb{R}^{(N-2)^2}$.

Note that with the five-point stencil, the matrix $A$ was never assembled and is nowhere in sight! 


### Numerical Solvers

To solve the linear system, two iterative methods are used and compared. 

$\textbf{Jacobi method}$

\begin{align*}
u^{(k+1)} = D^{-1}( f - Lu_{k} -Uu_k),
\end{align*}
where $D, L, U$ are the diagonal matrix, lower triangular matrix and upper triangular matrix of $A$, respectively.
Write into stecil,
\begin{align*}
u^{(k+1)}_{ij} = (h^2 f_{ij} + u^{(k)}_{i-1,j} +u^{(k)}_{i+1,j} + u^{(k)}_{i, j-1}+u^{(k)}_{i,j+1})/4 \qquad i,j = 1,\cdots, N-1 
\end{align*} 



$\textbf{Gauss-Seidel method}$ follows a similar scheme:

\begin{align*}
u^{(k+1)} = D^{-1} (f - L u_{k+1}- U u_k),
\end{align*}

Note that Gauss-Seidel method uses the most recent estimate to update.
 
Likewise, applying the Gauss-Seidel method doesn't require assembling $D, L, U$ matrices for finite-difference discretised Laplacian. 
Elementwise, we have 
\begin{align*}
u^{(k+1)}_{ij} = (h^2 f_{ij} + u^{(k)}_{i-1,j} +u^{(k+1)}_{i+1,j} + u^{(k+1)}_{i, j-1}+u^{(k)}_{i,j+1})/4 \qquad i,j = 1,\cdots, N-1 
\end{align*}

### Convergence Rate

Gauss Seidel method is known to be faster than Jacobi method (twice faster as stated in some textbooks), both of their convergence rate for our application is governed by the following theorem.

$\textbf{Theorem}\;$
The convergence rate of Jacobi and Gauss-Seidel method for the 5-point stencil finite difference method of the Poisson equation on a uniform mesh with size $h$ is
\begin{align*}
1- \mathcal{O}(h^2)
\end{align*}

As such, the convergence rate stalls as the mesh gets finer. To alleviate the shortcoming of the numerical method, let's try improving the perforance by OpenMP! 

### Exercise 4.0
Inspect the serial codebase [fd_laplace-serial.c](./fd_laplace-serial.c). Identify the two solvers **Jacobi** and **GaussSeidel**.

In [25]:
# Compile the serial code
!gcc -g -Wall -O3 -o fd_laplace-serial fd_laplace-serial.c -lm

In [26]:
# Execute the serial code with Jacobi method to solve on a grid of 300 x 300 meshes i.e. matrix size 90000 x 90000 at stopping criterion 1e-5.
!./fd_laplace-serial 300 1e-5 Jacobi

Jacobi METHOD IS IN USE
Residual after 1000 iteration: 2792.5315167908
Residual after 2000 iteration: 2642.5622881079
Residual after 3000 iteration: 2500.6469594138
Residual after 4000 iteration: 2366.3530066127
Residual after 5000 iteration: 2239.2711337460
Residual after 6000 iteration: 2119.0140255557
Residual after 7000 iteration: 2005.2151670396
Residual after 8000 iteration: 1897.5277264016
Residual after 9000 iteration: 1795.6234979903
Residual after 10000 iteration: 1699.1919020067
Residual after 11000 iteration: 1607.9390379313
Residual after 12000 iteration: 1521.5867887847
Residual after 13000 iteration: 1439.8719734940
Residual after 14000 iteration: 1362.5455447792
Residual after 15000 iteration: 1289.3718301166
Residual after 16000 iteration: 1220.1278134653
Residual after 17000 iteration: 1154.6024555670
Residual after 18000 iteration: 1092.5960507491
Residual after 19000 iteration: 1033.9196182693
Residual after 20000 iteration: 978.3943263470
Residual after 21000 itera

In [ ]:
#  Verify the results by plot
%run  'Laplace-plot.py'

### Exercise 4.1
Add appropriate work-sharing loop construct and clauses in the **Jacobi** function provided in [fd_laplace-omp.c](./fd_laplace-omp.c) to accelerate

In [ ]:
# Compile the parallel code

!gcc -fopenmp -g -Wall -O3 -o fd_laplace-omp fd_laplace-omp.c -lm

In [ ]:
# Execute the parallel code with Jacobi method to solve on a grid of 300 x 300 meshes i.e. matrix size 90000 x 90000 at stopping criterion 1e-5.

!OMP_NUM_THREADS=4 ./fd_laplace-omp 300 1e-5 Jacobi

In [ ]:
#  Verify the results by plotting
%run  'Laplace-plot.py'

Add appropriate work-sharing loop construct and clauses in the **GaussSeidel** function provided in [fd_laplace-omp.c](./fd_laplace-omp.c) to accelerate

In [ ]:
# Recompile the parallel code again if you changed the Gauss-Seidel method

!gcc -fopenmp -g -Wall -O3 -o fd_laplace-omp fd_laplace-omp.c -lm

In [ ]:
# Execute the parallel code with Gauss-Seidel method to solve on a grid of 300 x 300 meshes i.e. matrix size 90000 x 90000 at stopping criterion 1e-5.

!OMP_NUM_THREADS=4 ./fd_laplace-omp 300 1e-5 Gauss-Seidel 

In [ ]:
#  Verify the results by plot
%run  'Laplace-plot.py'

### Takeaway from Exercise 4

Don’t optimize prematurely. First choose most suitable primitive algorithms; then consider parallelisation and tuning.